In [ ]:
#install.packages(c("ape","ggplot2"))

In [1]:
library(ape)
library(ggplot2)

In [2]:

vdir=""#"../from_list_01/"
tdir=""#../fig1/concatenated_tree/
md="0807"
###Host diversity
nspl=read.csv(paste0(vdir,"viruslist-single-pan",md,"_plus2.tsv"),header=T,sep='\t')
colnames(nspl)[which(colnames(nspl)=='Lineage')]='virTree'
colnames(nspl)[which(colnames(nspl)=='Host_lineage')]='EukTree'

levels(nspl$virTree)=sort(as.character(sub(" ","_",sub(" $","",unique(nspl$virTree)))))
euktax=read.csv(paste0("euktax.tsv"),header=T,sep='\t')
#nspl=nspl[-which(nspl$EukTree==""),]
nspl[which(nspl$Family=='Mimiviridae-P'),'Family']='Mimiviridae'
nspl[which(nspl$Family=='Pandoraviridae'),'Family']='Phycodnaviridae'
nspl[which(nspl$Family=='Ascoviridae'),'Family']='Iridoviridae'

eutx=NULL
for(i in 1:nrow(euktax)){
  eutx[grep(euktax$EukTree[i],nspl$EukTree)]=as.character(euktax$hostLineage[i])
}
cspl=cbind(nspl,eutx)

# hdiv=aggregate(.~Family,cspl,function(x) c(mean=mean(x),sd=sd(x),len=length(unique(x))))
# #write.table(unique(spl$EukTree),paste0(vdir,"visualization/pair_compare/","euktax"),quote=F,row.names=F,col.names=F)
# plot(hdiv$eutx[,'len'],hdiv$Length[,'sd'])
# plot(hdiv$eutx[,'len'],hdiv$gf.unique.fm[,'mean'])
# 
# ggplot(hdiv,aes(x=hdiv$eutx[,'len'],y=hdiv$aanum[,'sd']))+
#   geom_point()+theme_classic()+ geom_text(aes(label=hdiv$Family)) 
panlin=read.csv(paste0(vdir,"pangenome",md,"lineage.tsv"),header=T,sep='\t')

# entropy
entt=function(ps){
  ps=ps[which(ps>0)]
  ans=0.000001
  for(i in 1:length(ps)){ 
    ans=ans+(ps[i]*log(ps[i]))
  }
  return(-ans)
}


# core gene seq div
library(ape)
tre1=read.tree(file=paste0(tdir,"iq2-allmaf_og.fasta.treefile"))
kalldist=NULL
for(i in 1:nrow(cspl)){
  kalldist[i]=cophenetic.phylo(tre1)['P49',cspl[i,'Abre']]
}
cspl=cbind(cspl,kalldist)


#abl=cspl[which(cspl$Family==hdiv2[i,'family']),'Abre']

corediv=function(abl){
  alldist=0
  for(j in abl){
    alldist=alldist+mean(cophenetic.phylo(tre1)[j,as.character(abl[-which(abl==j)])])
  }
  return(alldist/length(abl))
}


lincorediv=function(i){
  lineage_pool=unique(cspl$virTree[which(cspl$Family==unique(cspl$Family)[i])])
  lins2lins=0
  #abl=cspl$Abre[which(cspl$Family==unique(cspl$Family)[i])]
  for(v in lineage_pool){
    lin2lins=0
    lineage_pool=lineage_pool[-which(lineage_pool==v)]#c(used,as.character(cspl[which(cspl$virTree==v),'Abre'])
    if(length(lineage_pool)==0){break;}
    for(j in cspl[which(cspl$virTree==v),'Abre']){
      gm2lins=0
      for(k in lineage_pool){
        gm2lins=gm2lins+mean(cophenetic.phylo(tre1)[as.character(j),as.character(cspl[which(cspl$virTree==k),'Abre'])])
      }
      lin2lins=lin2lins+gm2lins/length(lineage_pool)
    }
    #lins2lins=lins2lins+lin2lins/length(cspl[which(cspl$virTree==v),'Abre'])
    lins2lins=c(lins2lins,lin2lins/length(cspl[which(cspl$virTree==v),'Abre']))
  }
  #lins2lins/(length(unique(cspl$virTree[which(cspl$Family==unique(cspl$Family)[i])]))-1)
  return(lins2lins[-1])
}



#lineage mean
linlin=function(i,colm){
  ans=NULL
  for(v in unique(cspl$virTree[which(cspl$Family==unique(cspl$Family)[i])])){
    ans=c(ans,mean(cspl[which(cspl$virTree==v),colm],na.rm=T))
  }
  return(ans)
}
lincat=function(i,colm){
  ans=NULL
  for(v in unique(cspl$virTree[which(cspl$Family==unique(cspl$Family)[i])])){
    ans=c(ans,as.character(unique(cspl[which(cspl$virTree==v),colm])))
  }
  return(ans)
}

get_panlin=function(i,colm){
  return(panlin[which(panlin$lineage %in% unique(cspl$virTree[which(cspl$Family==unique(cspl$Family)[i])])),colm])
}

hdiv2=cbind(family=as.character(unique(cspl[,'Family'])),gmlm=0,gmlsd=0,aanm=0,aansd=0,
            nhost=0,hosts=0,hdiv=0,ugpnm=0,ugpnsd=0,sgpnm=0,sgpnsd=0,
            kadisd=0,kodm=0,kodsd=0,ipsd=0,fmgpsd=0,sgt=0,
            coregp=0,corept=0,coreppp=0,coregpp=0)
for(i in 1:nrow(hdiv2)){
  cat(i)
  hdiv2[i,'gmlm']=mean(linlin(i,'Length'),na.rm=T)
  hdiv2[i,'gmlsd']=sd(linlin(i,'Length'),na.rm=T)
  hdiv2[i,'aanm']=mean(linlin(i,'aanum'),na.rm=T)
  hdiv2[i,'aansd']=sd(linlin(i,'aanum'),na.rm=T) # aa number
  hdiv2[i,'ugpnm']=mean(linlin(i,'gf.unique.sp'),na.rm=T) #
  hdiv2[i,'ugpnsd']=sd(linlin(i,'gf.unique.sp'),na.rm=T) # unique gp
  hdiv2[i,'sgpnm']=mean(linlin(i,'gf.shared'),na.rm=T)
  hdiv2[i,'sgpnsd']=sd(linlin(i,'gf.shared'),na.rm=T) # shared gp
  hdiv2[i,'kadisd']=sd(linlin(i,'kalldist'),na.rm=T)
  hdiv2[i,'kodm']=mean(lincorediv(i),na.rm=T) # core gene seq div
  hdiv2[i,'kodsd']=sd(lincorediv(i),na.rm=T)
  hdiv2[i,'ipsd']=sd(linlin(i,'inpara.pt'),na.rm=T) # inparalog
  hdiv2[i,'fmgpsd']=sd(linlin(i,'gf.unique.fm'),na.rm=T) # family unique gene
  hdiv2[i,'sgt']=sd(linlin(i,'ptng'),na.rm=T)#singleton
  
  hdiv2[i,'coregp']=sd(get_panlin(i,'core.clusters'),na.rm=T)
  hdiv2[i,'corept']=sd(get_panlin(i,'core.proteins'),na.rm=T)
  hdiv2[i,'coreppp']=mean(get_panlin(i,'core.proteins')/(get_panlin(i,'unique.allp')+get_panlin(i,'spec.lin.allp')+
                                get_panlin(i,'inter.lin.allp')+get_panlin(i,'inter.fm.allp')+get_panlin(i,'all.ptng')))
  hdiv2[i,'coregpp']=mean(get_panlin(i,'core.clusters')/(get_panlin(i,'unique.all')+get_panlin(i,'spec.lin.all')+
                                get_panlin(i,'inter.lin.all')+get_panlin(i,'inter.fm.all')))
  
  
  hdiv2[i,'hdiv']=entt(table(lincat(i,'eutx'))/sum(table(lincat(i,'eutx')),na.rm=T))
  hdiv2[i,'nhost']=length(unique(cspl[which(cspl$Family==hdiv2[i,'family']),'eutx']))
  hdiv2[i,'hosts']=paste(unique(cspl[which(cspl$Family==hdiv2[i,'family']),'eutx']),collapse=',')
}
hdiv2=as.data.frame(hdiv2)



pdf(paste0('hostdiv_',md,'b.pdf'),
    width=8.268/2.1,height=11.693/4.2,paper='special')
a=ggplot(hdiv2,aes(x=as.numeric(as.character(hdiv)),y=as.numeric(as.character(aansd))))+
  geom_point(size=2)+theme_classic()+ geom_text(aes(label=family),vjust=1.5)+
  labs(x="Supergroup-level host diversity", y="SD of protein numbers")+
  scale_x_continuous(limits=c(-0.1,1.8)) #protein number sd
print(a)

a=ggplot(hdiv2,aes(x=as.numeric(as.character(hdiv)),y=as.numeric(as.character(sgt))))+
  geom_point(size=2)+theme_classic()+ geom_text(aes(label=family),vjust=1.5)+
  labs(x="Supergroup-level host diversity", y="SD of singleton numbers")+
  scale_x_continuous(limits=c(-0.1,1.8)) #expand_limits(x=-0.5)
print(a)


a=ggplot(hdiv2,aes(x=as.numeric(as.character(hdiv)),y=as.numeric(as.character(kodm))))+
  geom_point(size=2)+theme_classic()+ geom_text(aes(label=family),vjust=1.5)+
  labs(x="Supergroup-level host diversity", y="Core-gene tree divergence")+
  scale_x_continuous(limits=c(-0.1,1.8))+expand_limits(y=0) #divergence mean
print(a)


dev.off()



1234567

png 
  2

In [ ]:
# a=ggplot(hdiv2,aes(x=as.numeric(as.character(hdiv)),y=as.numeric(as.character(coreppp))))+
#   geom_point(size=2)+theme_classic()+ geom_text(aes(label=family),vjust=1.5)+
#   labs(x="Diversity of eukaryote supergroup infected", y="Core gene number proportion")+
#   scale_x_continuous(limits=c(-0.1,1.8)) #expand_limits(x=-0.5)
# print(a)

# a=ggplot(hdiv2,aes(x=as.numeric(as.character(hdiv)),y=as.numeric(as.character(ugpnsd))))+
#   geom_point(size=2)+theme_classic()+ geom_text(aes(label=family),vjust=1.5)+
#   labs(x="Diversity of eukaryote supergroup infected", y="Unique gene cluster number variation")+
#   scale_x_continuous(limits=c(-0.1,1.8)) #expand_limits(x=-0.5)
# print(a)
# 
# a=ggplot(hdiv2,aes(x=as.numeric(as.character(hdiv)),y=as.numeric(as.character(ipsd))))+
#   geom_point(size=2)+theme_classic()+ geom_text(aes(label=family),vjust=1.5)+
#   labs(x="Diversity of eukaryote supergroup infected", y="Paralog number variation")+
#   scale_x_continuous(limits=c(-0.1,1.8)) #expand_limits(x=-0.5)
# print(a)



# a=ggplot(hdiv2,aes(x=as.numeric(as.character(hdiv)),y=as.numeric(as.character(gmlsd))))+
#   geom_point(size=2)+theme_classic()+ geom_text(aes(label=family),vjust=1.5)+
#   labs(x="Diversity of eukaryote supergroup infected", y="Genome length variation")+
#   scale_x_continuous(limits=c(-0.1,1.8)) #genome length sd
# print(a)

# a=ggplot(hdiv2,aes(x=as.numeric(as.character(hdiv)),y=as.numeric(as.character(coregp))))+
#   geom_point(size=2)+theme_classic()+ geom_text(aes(label=family),vjust=1.5)+
#   labs(x="Diversity of eukaryote supergroup infected", y="Core gene cluster variation")+
#   scale_x_continuous(limits=c(-0.1,1.8)) #expand_limits(x=-0.5)
# print(a)
# 
# a=ggplot(hdiv2,aes(x=as.numeric(as.character(hdiv)),y=as.numeric(as.character(corept))))+
#   geom_point(size=2)+theme_classic()+ geom_text(aes(label=family),vjust=1.5)+
#   labs(x="Diversity of eukaryote supergroup infected", y="Core gene number variation")+
#   scale_x_continuous(limits=c(-0.1,1.8)) #expand_limits(x=-0.5)
# print(a)
# 
# a=ggplot(hdiv2,aes(x=as.numeric(as.character(hdiv)),y=as.numeric(as.character(coregpp))))+
#   geom_point(size=2)+theme_classic()+ geom_text(aes(label=family),vjust=1.5)+
#   labs(x="Diversity of eukaryote supergroup infected", y="Core gene cluster proportion")+
#   scale_x_continuous(limits=c(-0.1,1.8)) #expand_limits(x=-0.5)
# print(a)
